In [1]:
import pandas as pd
import numpy as np

In [2]:
# dataTrain = 'https://drive.google.com/file/d/1s9koNFTgraCMbx0P2V-L_ogvA3map9PD/view?usp=sharing' # copy of original train dataset, in personal drive
# dataTest = 'https://drive.google.com/file/d/1d0zLt8IG2Icqmw4jwURPaDhkiWEsVn3n/view?usp=sharing' # copy of original test dataset, in personal drive
# dataTrainPath = 'https://drive.google.com/uc?export=download&id='+dataTrain.split('/')[-2]
# dataTestPath = 'https://drive.google.com/uc?export=download&id='+dataTest.split('/')[-2]
path = "../../data_worthcheck/"
dataTrain = pd.read_csv(path + "train.csv")
dataTest = pd.read_csv(path + "test.csv")

In [3]:
# Importing libraries
import tensorflow as tf
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [4]:
X_train = dataTrain['text_a'].values
y_train = dataTrain['label']

X_test = dataTest['text_a'].values
y_test = dataTest['label']

maps = {"no": 0, "yes": 1}
y_train = y_train.replace(maps)
y_test = y_test.replace(maps)


In [6]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
Xcnn_train = tokenizer.texts_to_sequences(X_train)
Xcnn_test = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1  
print(X_train[1])
print(Xcnn_train[1])

mas piyuuu mugo2 corona tuh mulut tersumpal ma corona
[158, 823, 1, 129, 1031, 520, 1]


In [7]:
# Padding the data samples to a maximum review length in words
max_words = 450
Xcnn_train = sequence.pad_sequences(Xcnn_train, maxlen=max_words,  padding='post')
Xcnn_test = sequence.pad_sequences(Xcnn_test, maxlen=max_words,  padding='post')
# Building the CNN Model
model = Sequential()      # initilaizing Sequential for CNN 
# Adding the embedding layer which will take in maximum of 450 words as input and provide a 32 dimensional output 
model.add(Embedding(vocab_size, 32, input_length=max_words))
# Conv Layer
model.add(Conv1D(32, 3, padding='same', activation='relu'))
# Pooling Layer
model.add(MaxPooling1D())
model.add(Flatten())
# Dense Layer
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [8]:
# Optimize with SGD
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.1), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 450, 32)           1482080   
                                                                 
 conv1d (Conv1D)             (None, 450, 32)           3104      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 225, 32)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 7200)              0         
                                                                 
 dense (Dense)               (None, 256)               1843456   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                        

In [9]:
# Fitting the model and search f1 score
from sklearn.metrics import f1_score
model.fit(Xcnn_train, y_train,
                    epochs=3,
                    verbose=False,
                    validation_data=(Xcnn_test, y_test),
                    batch_size=16)
loss, accuracy = model.evaluate(Xcnn_train, y_train, verbose=False)
print("Training Accuracy: {:.6f}".format(accuracy))
loss, accuracy = model.evaluate(Xcnn_test, y_test, verbose=False)
print("Testing Accuracy:  {:.6f}".format(accuracy)) 


yhat_probs = model.predict(Xcnn_test, verbose=0)
predict_y = model.predict(Xcnn_test, verbose=0)
yhat_classes=np.argmax(predict_y,axis=1)
yhat_probs = yhat_probs[:, 0]
 

f1 = f1_score(y_test, yhat_classes, average='micro')
print('F1 score: %f' % f1)

Training Accuracy: 0.908060
Testing Accuracy:  0.853929
F1 score: 0.747500
